In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os

import keras.models as models
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

from keras import backend as K

import cv2
import numpy as np
import json
np.random.seed(7) # 0bserver07 for reproducibility
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
data_shape = 360*480

class_weighting= [0.2595, 0.1826, 4.5640, 0.1417, 0.5051, 0.3826, 9.6446, 1.8418, 6.6823, 6.2478, 3.0, 7.3614]


# load the model:
with open('segNet_basic_model.json') as model_file:
    segnet_basic = models.model_from_json(model_file.read())

# Compile model (required to make predictions)
segnet_basic.compile(loss="categorical_crossentropy", optimizer='adadelta', metrics=["accuracy"])


In [3]:
segnet_basic.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_1 (Layer)              (None, 360, 480, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 360, 480, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 360, 480, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 360, 480, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 180, 240, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 180, 240, 128)     73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 180, 240, 128)     512       
__________

In [4]:
# load the data
train_data = np.load('./data/train_data.npy')
train_label = np.load('./data/train_label.npy')

In [ ]:
valid_data = np.load('./data/val_data.npy')
valid_label = np.load('./data/val_label.npy')

In [ ]:

test_data = np.load('./data/test_data.npy')
test_label = np.load('./data/test_label.npy')

In [ ]:
history = segnet_basic.fit(train_data, train_label, batch_size=3, epochs=15,
                    verbose=1, class_weight=class_weighting , validation_data=(valid_data, valid_label),
                    shuffle=True) # validation_split=0.33

In [ ]:
batch_size = 6

# estimate accuracy on whole dataset using loaded weights
# load weights
#segnet_basic.load_weights("weights.best.hdf5")
scores = segnet_basic.evaluate(test_data, test_label, verbose=0, batch_size=batch_size)
print("%s: %.2f%%" % (segnet_basic.metrics_names[1], scores[1]*100))

In [ ]:
Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road_marking = [255,69,0]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

label_colours = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

def visualize(temp, plot=True):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0,11):
        r[temp==l]=label_colours[l,0]
        g[temp==l]=label_colours[l,1]
        b[temp==l]=label_colours[l,2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = (r/255.0)#[:,:,0]
    rgb[:,:,1] = (g/255.0)#[:,:,1]
    rgb[:,:,2] = (b/255.0)#[:,:,2]
    if plot:
        plt.imshow(rgb)
    else:
        return rgb


In [ ]:
import os
DataPath = './CamVid/'

gt = []
with open(DataPath+'train.txt') as f:
    txt = f.readlines()
    txt = [line.split(' ') for line in txt]
for i in range(len(txt)):
    gt.append(cv2.imread(os.getcwd() + txt[i][0][7:]))


output = segnet_basic.predict_proba(train_data[23:24])
pred = visualize(np.argmax(output[0],axis=1).reshape((360,480)), False)
plt.imshow(pred)
plt.figure(2)
plt.imshow(gt[23])

In [ ]:
output = segnet_basic.predict_proba(train_data[20:21])
pred = visualize(np.argmax(output[0],axis=1).reshape((360,480)), False)
plt.imshow(pred)
plt.figure(2)
plt.imshow(gt[20])

In [ ]:
output = segnet_basic.predict_proba(train_data[10:11])
pred = visualize(np.argmax(output[0],axis=1).reshape((360,480)), False)
plt.imshow(pred)
plt.figure(2)
plt.imshow(gt[10])

In [ ]:
# choose any image to want by specifying the index
img_to_visualize = X_train[22:23]
# Keras requires the image to be in 4D
# So we add an extra dimension to it.
img_to_visualize = np.expand_dims(img_to_visualize, axis=0)

In [ ]:
def layer_to_visualize(layer):
    inputs = [K.learning_phase()] + segnet_basic.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + [X])

    convolutions = convout1_f(img_to_visualize)
    convolutions = np.squeeze(convolutions)

    print ('Shape of conv:', convolutions.shape)
    
    n = convolutions.shape[0]
    n = int(np.ceil(np.sqrt(n)))
    
    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(12,8))
    for i in range(len(convolutions)):
        ax = fig.add_subplot(n,n,i+1)
        ax.imshow(convolutions[i], cmap='gray')

In [ ]:

# Specify the layer to want to visualize
layer_to_visualize(convout1)

# As convout2 is the result of a MaxPool2D layer
# We can see that the image has blurred since
# the resolution has reduced 
layer_to_visualize(convout2)